In [1]:
import os

In [2]:
# Store your API keys in a common file and reference it here
from apikeys import OPEN_AI_KEY, DEEP_LAKE_KEY

In [3]:
os.environ["OPENAI_API_KEY"] = OPEN_AI_KEY

## The LLMs

With LangChain, we can call a LLM on our prompt, we are going to do that right now. I am going to ask the LLM to give me a top 10 tips for efficiently managing time. 

In [4]:
from langchain.llms import OpenAI

The `temperature` parameter determines the randomness, `0` is stable and `1` is random. Creative tasks should be in the range of `0.7` and `0.9`. 

In [6]:
llm_davinci = OpenAI(model_name = "text-davinci-003", temperature= 0.9)

In [7]:
prompt = "Give me the top 10 tips for efficiently managing time"

In [9]:
tips = llm_davinci(prompt)

In [10]:
print(tips)



1. Prioritize your tasks – understand what is most important to get done and focus on completing those things first.

2. Create a plan – map out each day, week, and month so you know what needs to be accomplished and when.

3. Eliminate distractions – turn off notifications and focus on one task at a time to stay on track.

4. Tackle the difficult tasks first – doing the hard tasks when you have the most energy will help you stay motivated throughout the day.

5. Break down large projects – breaking down larger projects into smaller chunks makes them more manageable and allows for better progress tracking.

6. Utilize task management tools – there are many tools available to help manage tasks and keep track of progress.

7. Take frequent breaks – working non-stop is not effective – taking short breaks throughout the day can help you stay focused and productive.

8. Set realistic goals – setting achievable goals helps you stay motivated and focused on the tasks at hand.

9. Outsource 